# MCMC4: Replica Exchange Monte Carlo (Parallel Tempering)

In order to get correct and unbiased results from QMC simulations, we need to implement a "global updating" algorithm in addition to the single-flip "local updating." Here I try to rewrite the previous code using Channels to implement parallelization to show how it goes in Jupyter Notebook. However, in most cases ResumableFunctions.jl + MPI.jl is much faster, and you should use MPI.jl instead of using Channels.